#Качаем датасет

In [4]:
!wget -O pred_data_crop_minCSV.zip https://www.dropbox.com/s/5lkgz3zw9t78i8c/pred_data_crop_minCSV.zip?dl=0

In [5]:
!unzip pred_data_crop_minCSV.zip

#Библиотеки

In [6]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Normalization
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomCrop, CenterCrop, RandomZoom, RandomContrast
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import utils
from tensorflow.keras.applications.vgg16 import preprocess_input as VGG16_ResNet50_preprocess
import matplotlib.pyplot as plt
# from google.colab import files
%matplotlib inline 
from PIL import Image

#Обрабокта данных

In [7]:
w = 256
h = 256

In [8]:
def pred(img):
    i = Image.open(img)
    i = i.resize((256, 256), Image.ANTIALIAS)
    return np.array(i)

In [9]:
train = pd.read_csv('train.csv')
train

In [10]:
x_train = train["Названия"].apply(pred)
x_train = np.array([np.array(val,dtype=np.float32) for val in x_train])

In [11]:
x_train=VGG16_ResNet50_preprocess(x_train)

In [12]:
y_train = train.iloc[:,1:]
y_train

#Расширение данных

In [13]:
data_augmentation = Sequential()
data_augmentation.add(RandomFlip("vertical", input_shape=(256,256,3)))
data_augmentation.add(RandomRotation(0.2))
data_augmentation.add(RandomZoom(0.3))
data_augmentation.add(RandomCrop(128, 128))

#Предобученная сеть

In [14]:
count_klass = y_train.shape[1]

In [15]:
ResNet50_net = ResNet50(weights='imagenet', 
                  include_top=False, 
                  input_shape=(128, 128, 3))

In [16]:
ResNet50_net.trainable = False

In [17]:
model = Sequential()

model.add(data_augmentation)

# Добавляем модель VGG16 в сеть как слой
model.add(ResNet50_net)

model.add(Flatten())

model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

# model.add(Dropout(0.5))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(2048, activation='relu'))
model.add(Dense(count_klass, activation='sigmoid'))

**Компилируем модель**

In [18]:
model.summary()

#Обратный вызов

#Обучаем нейронную сеть

In [19]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + 
    K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_m])

In [20]:
ch_p=ModelCheckpoint('best_model.h5', 
                    monitor='val_f1_m',
                    mode='max',
                    verbose=1, 
                    save_best_only=True,
                    )

In [21]:
# e_s=EarlyStopping(monitor='val_f1_m', 
#                       patience=15, 
#                       verbose=1
#                       )


In [22]:
model.fit(x_train, y_train,
          validation_split=0.2, 
          epochs=500, 
          batch_size=10,
          callbacks=[ch_p],
          verbose=1)

#Оцениваем качество обучения

In [41]:
test = pd.read_csv('test.csv')
x_test = test["Названия"].apply(pred)
x_test = np.array([np.array(val,dtype=np.float32) for val in x_test])
x_test=VGG16_ResNet50_preprocess(x_test)
y_test = test.iloc[:,1:]
y_test

In [42]:
 model.load_weights("best_model.h5")

In [43]:
predictions = model.predict(x_test)


In [ ]:
# n = 10
# plt.imshow(x_test[n], cmap=plt.cm.binary)
# plt.show()
# # print(predictions[n].round())
# # print(np.round(predictions[n], 3))
# arr =[]
# for i in predictions[n]:
#     if i >= 0.5:
#       arr.append(1)
#     else:
#       arr.append(0)
# print(arr)
# print(y_test.iloc[n])

In [44]:
model.evaluate(x_test, y_test)

#Тонкая настройка

In [29]:
ResNet50_net.trainable = True
trainable = False
for layer in ResNet50_net.layers:
    if layer.name == 'block5_conv1':
        trainable = True
    layer.trainable = trainable

In [30]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=1e-5), 
              metrics=[f1_m])

In [32]:
history = model.fit(x_train, y_train,
                    validation_split=0.2,
                    batch_size=10,
                    callbacks=[ch_p],
                    epochs=10)

In [34]:
model.load_weights("best_model.h5")

#Оцениваем качество обучения после тонкой настройки

In [36]:
predictions = model.predict(x_test)

In [37]:
model.evaluate(x_test,y_test)

In [45]:
n = 10
plt.imshow(x_test[n], cmap=plt.cm.binary)
plt.show()
# print(predictions[n].round())
# print(np.round(predictions[n], 3))
arr =[]
for i in predictions[n]:
    if i >= 0.5:
      arr.append(1)
    else:
      arr.append(0)
print(arr)
print(y_test.iloc[n])